# DSCI 100 Group 2 Project Proposal

## Heart Failure Prediction




### Introduction

Cardiovascular diseases (CVDs) are the largest contributor to death globally (about 31%, or 17.9 million lives per year). This data set observes a range of variables related to the heart that could potentially predict a heart disease, including cholesterol levels, types of chest pain, blood pressure and sugar, as well as variables regarding age, sex, and the presence or absence of heart disease. This project will attempt to answer the following predictive question: what variable(s) most strongly predict(s) presence of a heart disease?

The dataset we will be using, which is found on Kaggle [here][1], is the combination of five different datasets within the UCI Machine Learning Repository’s Heart Disease Data Set found [here][2]. Although the dataset contains 76 attributes, we will be using a subset of 12 that other published experiments have used as well. The attributes are described as follows:

[1]: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction
[2]: https://archive.ics.uci.edu/ml/datasets/Heart+Disease

1. Age: age of patient (years)
2. Sex: sex (M = male, F = female)
3. RestingBP: resting blood pressure (mm/Hg)
4. Cholesterol: Serum cholesterol (mg/dl)
5. FastingBS: fasting blood sugar (1 if FastingBS > 120 mg/dl, otherwise 0)
6. MaxHR: maximum heart rate achieved (value between 60 and 202)
7. ExerciseAngina: exercise induced angina (Y = Yes, N = No)
8. HeartDisease: output/prediction (1 = heart disease, 0 = normal)

### Preliminary Exploratory Data Analysis

In [1]:
import altair as alt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
url = "https://raw.githubusercontent.com/caojason/dsci-100-group-2/main/data/heart.csv"
heart_data = pd.read_csv(url)
heart_data

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [3]:
heart_data_filtered = heart_data[
    ["Age","Sex","RestingBP","Cholesterol","FastingBS","MaxHR","ExerciseAngina","HeartDisease"]
]

heart_data_filtered
heart_data_nozeroes = heart_data_filtered[
    (heart_data["Cholesterol"] > 0) &
    (heart_data["RestingBP"] > 0) &
    (heart_data["MaxHR"] > 0)
]
heart_data_nozeroes

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,HeartDisease
0,40,M,140,289,0,172,N,0
1,49,F,160,180,0,156,N,1
2,37,M,130,283,0,98,N,0
3,48,F,138,214,0,108,Y,1
4,54,M,150,195,0,122,N,0
...,...,...,...,...,...,...,...,...
913,45,M,110,264,0,132,N,1
914,68,M,144,193,1,141,N,1
915,57,M,130,131,0,115,Y,1
916,57,F,130,236,0,174,N,1


In [4]:
heart_data_nozeroes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 746 entries, 0 to 917
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Age             746 non-null    int64 
 1   Sex             746 non-null    object
 2   RestingBP       746 non-null    int64 
 3   Cholesterol     746 non-null    int64 
 4   FastingBS       746 non-null    int64 
 5   MaxHR           746 non-null    int64 
 6   ExerciseAngina  746 non-null    object
 7   HeartDisease    746 non-null    int64 
dtypes: int64(6), object(2)
memory usage: 52.5+ KB


We can see that our filtered dataset has 746 observations for each attribute and missing data has been removed.

In [5]:
heart_data_training, heart_data_testing = train_test_split(
    heart_data_nozeroes, test_size=0.25, random_state=1234
)
heart_data_training

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,HeartDisease
594,58,M,160,256,1,113,Y,1
824,37,M,130,250,0,187,N,0
199,57,F,130,308,0,98,N,0
62,45,M,140,224,0,144,N,0
730,49,F,134,271,0,162,N,0
...,...,...,...,...,...,...,...,...
544,61,F,140,298,1,120,Y,0
204,56,M,130,184,0,100,N,0
53,41,F,130,245,0,150,N,0
417,44,M,130,209,0,127,N,0


In [6]:
chol_plot = (
    alt.Chart(heart_data_training)
    .mark_bar()
    .encode(
        x=alt.X("Cholesterol",bin=alt.Bin(maxbins=30),title="Cholesterol (mg/dl)",scale=alt.Scale(zero=False)),
        y=alt.Y("count(HeartDisease)",title="Number of Heart Diagnoses"),
        fill=alt.Fill("HeartDisease:N", title="Heart Disease")
    )
    .configure_axis(titleFontSize=20,labelFontSize=16)

)

In [7]:
chol_plot

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

As seen in the plot above, in patients with moderate levels of cholesterol, heart disease is not seemingly strongly correlated with cholesterol (there are roughly the same number of diseased and not diseased patients with average cholesterol levels). However, in higher cholesterols (greater than 320mg/dl especially), there is a potential connection between cholesterol and the presence of heart disease. For this reason, serum cholesterol will be included in the classifier.

In [8]:
restbp_plot = (
    alt.Chart(heart_data_training)
    .mark_bar()
    .encode(
        x=alt.X("RestingBP",bin=alt.Bin(maxbins=30),title="Resting Blood Pressure (mm/Hg)",scale=alt.Scale(zero=False)),
        y=alt.Y("count(HeartDisease)",title="Number of Heart Diagnoses"),
        fill=alt.Fill("HeartDisease:N", title="Heart Disease")
    )
    .configure_axis(titleFontSize=20,labelFontSize=16)

)

In [9]:
restbp_plot

alt.Chart(...)

As seen in the plot above, in patients with moderate resting blood pressures, heart disease is not seemingly strongly correlated with resting blood pressures (there are roughly the same number of diseased and not diseased patients with average resting blood pressures levels). However, in higher resting blood pressures (greater than 135mm/Hg especially), there is a potential connection between resting blood pressures and the presence of heart disease. For this reason, resting blood pressures will be included in the classifier.

In [10]:
fastbs_plot = (
    alt.Chart(heart_data_training)
    .mark_bar()
    .encode(
        x=alt.X("FastingBS:N", bin=alt.Bin(maxbins=2), title="Fasting Blood Sugar > 120mg/dl?"),
        y=alt.Y("count(HeartDisease)",title="Number of Heart Diagnoses"),
        fill=alt.Fill("HeartDisease:N", title="Heart Disease")
    )
    .configure_axis(titleFontSize=20,labelFontSize=16)
)
fastbs_plot

alt.Chart(...)

As seen in the plot above, there seems to be a connection between high blood sugar and presence of heart disease. There seems to be a higher proportion of heart disease when the patient's fasting blood sugar is higher than 120mg/dl than when it is under 120mg/dl. For this reason, Fasting Blood Sugar will be included in the classifier.

In [11]:
maxhr_plot = (
    alt.Chart(heart_data_training)
    .mark_bar()
    .encode(
        x=alt.X("MaxHR",bin=alt.Bin(maxbins=30),title="Maximum Heart Rate (bpm)",scale=alt.Scale(zero=False)),
        y=alt.Y("count(HeartDisease)",title="Number of Heart Diagnoses"),
        fill=alt.Fill("HeartDisease:N", title="Heart Disease")
    )
    .configure_axis(titleFontSize=20,labelFontSize=16)

)
maxhr_plot

alt.Chart(...)

As seen in the plot above, there seems to be a connection between maximum heart rate and presence of heart disease. It seems that the higher the maximum heart rate, the lower the proportion of patients afflicted with heart disease. So, generally, it seems that those with heart diseases have lower maximum heart rates. For this reason, maximum heart rate will be included in the classifier.

### Methods

As our predictive question is a classification problem (classify a patient to have heart disease or not), we plan to use the K-nearest neighbors classification algorithm. We will be tuning the classifier using cross_validation and GridSearchCV in order to optimize for the K value. The model will be evaluated on the testing data, which is 25% of the original data, and we will be analyzing the score and the confusion matrix. We will be visualizing a graph of the mean test score vs K values.

Regarding our choice of predictor variables, we have selected to use RestingBP, Cholesterol, FastingBS, and MaxHR due to their apparent correlation with Heart Disease as seen in our EDA. For the variables Age, Sex, and ExerciseAngia, we have found professional sources that suggest a possible link between these variables and heart disease. These findings are listed below with their corresponding citation.

**Age:** Age is an independent risk factor for heart diseases (Jennifer L. Rodgers et al., 2019)

**Sex:** Women have been shown to be more susceptible to cardiovascular diseases. Factors like serum cholesterol, blood pressure seemingly increase with age in women (Pekka Jousilahti et al., 1999).

**Exercise angina:** The occurrence of acute angina and sudden death from CVD-related issues is highest in those who are not physically active (Paul D. Thompson et al., 2007).


### Expected Outcomes and Significance

We expect to be able to predict the existence of heart disease in a patient with significantly more accuracy when compared to using a majority classifier.

The impact of such findings will be significant. Cardiovascular diseases are the largest contributor to global deaths; being able to accurately predict their presence could help those afflicted get treatment earlier. This would also free up valuable time that doctors would have to spend to diagnose patients.

Future questions that may result from this project include determining which subset of the 76 predictor variables are the most important for heart disease prediction. This can also lead to the development of models that can give early warnings to at-risk individuals.

### References

* Jousilahti, P., Vartiainen, E., Tuomilehto , J., & Puska, P. (1999, March 9). Sex, age, cardiovascular risk factors, and coronary heart disease. Retrieved from  https://www.ahajournals.org/doi/10.1161/01.CIR.99.9.1165
* Rodgers, J. L., Jones, J., Bolleddu, S. I., Vanthenapalli, S., Rodgers, L. E., Shah, K., Karia, K., & Panguluri, S. K. (2019, April 27). Cardiovascular risks associated with gender and aging. Journal of cardiovascular development and disease. Retrieved March 11, 2023, from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6616540/
* Thompson, P. D., Franklin, B. A., Balady, G. J., Blair, S. N., Corrado, D., EstesIII, N. A. M., Fulton, J. E., Gordon, N. F., Haskell, W. L., Link, M. S., Maron, B. J., Mittleman, M. A., Pelliccia, A., Wenger, N. K., Willich, S. N., & Costa, F. (2007, April 27). Exercise and acute cardiovascular events | circulation. Retrieved March 11, 2023, from https://www.ahajournals.org/doi/10.1161/CIRCULATIONAHA.107.181485